 =============================================================
### 01 - Exploratory Data Analysis
 =============================================================

In [1]:
# --- 1. Imports & Setup ---
import pandas as pd
import html

In [2]:
# --- 2. Load Data ---

ds1 = pd.read_csv("../data/1429_1.csv")
ds2 = pd.read_csv("../data/amazon_customer_reviews_2017_2018.csv")
ds3 = pd.read_csv("../data/amazon_customer_reviews_Feb_April_2019.csv")

print(ds1.shape)
print(ds1.dtypes)
print("\n")
print(ds2.shape)
print(ds2.dtypes)
print("\n")
print(ds3.shape)
print(ds3.dtypes)

C:\Users\Work\AppData\Local\Temp\ipykernel_8824\3553111191.py:3: DtypeWarning: Columns (0: name, 1: reviews.didPurchase) have mixed types. Specify dtype option on import or set low_memory=False.
  ds1 = pd.read_csv("../data/1429_1.csv")


(34660, 21)
id                          str
name                        str
asins                       str
brand                       str
categories                  str
keys                        str
manufacturer                str
reviews.date                str
reviews.dateAdded           str
reviews.dateSeen            str
reviews.didPurchase      object
reviews.doRecommend      object
reviews.id              float64
reviews.numHelpful      float64
reviews.rating          float64
reviews.sourceURLs          str
reviews.text                str
reviews.title               str
reviews.userCity        float64
reviews.userProvince    float64
reviews.username            str
dtype: object


(5000, 24)
id                         str
dateAdded                  str
dateUpdated                str
name                       str
asins                      str
brand                      str
categories                 str
primaryCategories          str
imageURLs                  str
keys      

### Visualization of the datasets

In [ ]:
ds1.head()

In [ ]:
ds1.keys()

In [ ]:
ds2.head()

In [ ]:
ds2.keys()

In [ ]:
ds3.head()

In [ ]:
ds3.keys()

In [ ]:
# See the Electronics products in detail
electronics_ds3 = ds3[ds3["primaryCategories"] == "Electronics"]
print(electronics_ds3["name"].value_counts().head(30))

# Also check data_01 product names since it's all electronics
print(ds1["name"].value_counts().head(30))

# And review counts per product in Electronics
# (thin review counts = thin generation material)
print(electronics_ds3.groupby("name")["reviews.text"].count().sort_values(ascending=False).head(20))

### Decision on meta-categories

Rather than forcing artificial diversity across low-signal categories, we made a deliberate choice to go deep within Electronics — a domain with sufficient product breadth and review volume to produce meaningful generation output. Our 5 sub-categories reflect natural product families that consumers actually compare when making purchase decisions, mirroring how sites like The Wirecutter structure their buying guides

### The Cleaning Tasks Per Column

```python
KEEP = {
    "name"            → name_clean       # product identity for clustering/generation
    "reviews.rating"  → rating           # sentiment labels
    "reviews.text"    → review_text      # the core input for all 3 models
    "primaryCategories" → primary_category  # DS2/DS3 only, clustering reference
    "brand"           → brand            # useful context for generation
}
```

---

**`reviews.text` (most important)**

- Drop nulls
- Drop reviews under 20 characters (no signal)
- Strip leading/trailing whitespace
- Remove the \r\n duplication artifact (same as names)
- Decode any HTML entities (&amp; → &, etc.)


**`name`**

- Split on \r\n, take first part only
- Strip trailing commas and whitespace
- Drop nulls (can't assign category without a name)


**`reviews.rating`**

- Coerce to numeric, errors → NaN
- Drop NaN
- Cast to int
- Drop anything outside 1–5 range (data corruption)


**`primaryCategories` (DS2/DS3 only)**

- Strip whitespace
- Standardize multi-label separator (some use comma, some use comma+space)
- Keep as-is otherwise


**`brand`**

- Strip whitespace
- Fill nulls with "Unknown" — don't drop rows over a missing brand


In [3]:
# Load a dataset 

def load_raw(path: str, low_memory: bool = False) -> pd.DataFrame:
    """Load a raw CSV file into a DataFrame."""
    return pd.read_csv(path, low_memory=low_memory)

In [4]:
# Select and Rename

def select_columns(df: pd.DataFrame, has_primary_cat: bool = False) -> pd.DataFrame:
    """
    Keep only relevant columns and rename them to standard names.
    Gracefully skips columns that don't exist in this dataset.
    """
    keep = ["name", "brand", "reviews.rating", "reviews.text"]
    if has_primary_cat:
        keep.append("primaryCategories")

    # Only keep columns that actually exist
    existing = [c for c in keep if c in df.columns]
    df = df[existing].copy()

    return df.rename(columns={
        "reviews.text":      "review_text",
        "reviews.rating":    "rating",
        "primaryCategories": "primary_category",
    })

In [5]:
# Individual column cleaners

def clean_name(df: pd.DataFrame) -> pd.DataFrame:
    """Clean product name: remove duplication artifact, strip junk."""
    df["name"] = df["name"].apply(
        lambda x: "" if not isinstance(x, str) else x
    )
    df["name"] = (
        df["name"]
        .str.split("\r\n").str[0]
        .str.strip(" ,")
    )
    return df[df["name"].str.len() > 0].copy()

def clean_review_text(df: pd.DataFrame) -> pd.DataFrame:
    """Clean review text: remove artifacts, decode HTML, drop short reviews."""
    # Convert to string first, before any chained operation
    df["review_text"] = df["review_text"].apply(
        lambda x: "" if not isinstance(x, str) else x
    )
    df["review_text"] = (
        df["review_text"]
        .str.split("\r\n").str[0]
        .str.strip()
        .apply(html.unescape)
    )
    df = df[df["review_text"].str.len() >= 20].copy() # Remove any review under 20 characters due to low-signal 
    return df

def clean_brand(df: pd.DataFrame) -> pd.DataFrame:
    """Normalize brand: fill nulls, strip whitespace."""
    if "brand" not in df.columns:
        return df
    df["brand"] = df["brand"].fillna("Unknown").astype(str).str.strip()
    return df

def clean_primary_category(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize primary_category: fill nulls, strip whitespace,
    standardize comma separator.
    Handles float NaN values safely.     ← this was your bug
    """
    if "primary_category" not in df.columns:
        return df
    df["primary_category"] = (
        df["primary_category"]
        .fillna("Unknown")      # kills float NaN before any string ops
        .astype(str)            # ensures everything is a string
        .str.strip()
        .str.replace(", ", ",", regex=False)
    )
    return df

def clean_rating(df: pd.DataFrame) -> pd.DataFrame:
    """Coerce rating to int, drop nulls and out-of-range values."""
    df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
    df = df.dropna(subset=["rating"])
    df["rating"] = df["rating"].astype(int)
    return df[df["rating"].between(1, 5)].copy()

def clean_primary_category(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize primary_category: fill nulls, strip whitespace,
    standardize comma separator.
    Handles float NaN values safely.     ← this was your bug
    """
    if "primary_category" not in df.columns:
        return df
    df["primary_category"] = (
        df["primary_category"]
        .fillna("Unknown")      # kills float NaN before any string ops
        .astype(str)            # ensures everything is a string
        .str.strip()
        .str.replace(", ", ",", regex=False)
    )
    return df

In [6]:
# Main function to call helper functions

def load_and_clean(
    path: str,
    has_primary_cat: bool = False,
    low_memory: bool = False
) -> pd.DataFrame:
    """
    Full pipeline: load → select → clean each column → reset index.
    """
    df = load_raw(path, low_memory=low_memory)
    df = select_columns(df, has_primary_cat=has_primary_cat)
    df = clean_name(df)
    df = clean_review_text(df)
    df = clean_rating(df)
    df = clean_brand(df)
    df = clean_primary_category(df)
    return df.reset_index(drop=True)

In [7]:
# ── DS1 ──────────────────────────────────────────────────────
raw_ds1 = load_raw("../data/1429_1.csv", low_memory=False)
df1 = select_columns(raw_ds1, has_primary_cat=False)
df1 = clean_name(df1)
df1 = clean_review_text(df1)
df1 = clean_rating(df1)
df1 = clean_brand(df1)
ds1_clean = df1.reset_index(drop=True)
print(f"DS1: {len(ds1_clean):,} rows")

# ── DS2 ──────────────────────────────────────────────────────
raw_ds2 = load_raw("../data/amazon_customer_reviews_2017_2018.csv")
df2 = select_columns(raw_ds2, has_primary_cat=True)
df2 = clean_name(df2)
df2 = clean_review_text(df2)
df2 = clean_rating(df2)
df2 = clean_brand(df2)
df2 = clean_primary_category(df2)
ds2_clean = df2.reset_index(drop=True)
print(f"DS2: {len(ds2_clean):,} rows")

# ── DS3 ──────────────────────────────────────────────────────
raw_ds3 = load_raw("../data/amazon_customer_reviews_Feb_April_2019.csv")
df3 = select_columns(raw_ds3, has_primary_cat=True)
df3 = clean_name(df3)
df3 = clean_review_text(df3)
df3 = clean_rating(df3)
df3 = clean_brand(df3)
df3 = clean_primary_category(df3)
ds3_clean = df3.reset_index(drop=True)
print(f"DS3: {len(ds3_clean):,} rows")

# ── Summary ──────────────────────────────────────────────────
print("\n── Final clean row counts ──")
for name, dataset in [("DS1", ds1_clean), ("DS2", ds2_clean), ("DS3", ds3_clean)]:
    print(f"{name}: {len(dataset):,} rows | columns: {dataset.columns.tolist()}")

DS1: 27,747 rows
DS2: 5,000 rows
DS3: 25,794 rows

── Final clean row counts ──
DS1: 27,747 rows | columns: ['name', 'brand', 'rating', 'review_text']
DS2: 5,000 rows | columns: ['name', 'brand', 'rating', 'review_text', 'primary_category']
DS3: 25,794 rows | columns: ['name', 'brand', 'rating', 'review_text', 'primary_category']


In [8]:
import os
os.makedirs("../data/processed", exist_ok=True)

# ── Check 1: Sentiment corpus (all three combined) ────────────────────────
sentiment_df = pd.concat(
    [ds1_clean, ds2_clean, ds3_clean], ignore_index=True
).drop_duplicates(subset=["review_text"])

# Map ratings to sentiment labels
def rating_to_sentiment(rating: int) -> str:
    if rating >= 4:
        return "positive"
    elif rating == 3:
        return "neutral"
    return "negative"

sentiment_df["sentiment"] = sentiment_df["rating"].apply(rating_to_sentiment)

print("── Sentiment Corpus ──")
print(f"Total reviews: {len(sentiment_df):,}")
print(f"Class distribution:\n{sentiment_df['sentiment'].value_counts()}")
print(f"Class balance (%):\n{sentiment_df['sentiment'].value_counts(normalize=True).mul(100).round(1)}")

# ── Check 2: Electronics corpus (clustering + generation) ─────────────────
electronics_df = pd.concat([
    ds1_clean[["name", "brand", "rating", "review_text"]],
    ds3_clean[ds3_clean["primary_category"].str.contains("Electronics", na=False)]
             [["name", "brand", "rating", "review_text"]]
], ignore_index=True).drop_duplicates(subset=["review_text"])

print("\n── Electronics Corpus ──")
print(f"Total reviews: {len(electronics_df):,}")
print(f"Unique products: {electronics_df['name'].nunique()}")

# ── Check 3: Echo vs Fire TV split decision ───────────────────────────────
for keywords, label in [
    (["Echo", "Tap"],       "Echo / Tap"),
    (["Fire TV", "FireTV"], "Fire TV")
]:
    count = electronics_df["name"].str.contains(
        "|".join(keywords), case=False, na=False
    ).sum()
    print(f"{label}: {count:,} reviews")

# ── Save processed files ──────────────────────────────────────────────────
sentiment_df.to_csv("../data/processed/sentiment_ready.csv", index=False)
electronics_df.to_csv("../data/processed/electronics_ready.csv", index=False)
print("\n── Saved ──")
print("sentiment_ready.csv")
print("electronics_ready.csv")

── Sentiment Corpus ──
Total reviews: 39,794
Class distribution:
sentiment
positive    36273
neutral      1845
negative     1676
Name: count, dtype: int64
Class balance (%):
sentiment
positive    91.2
neutral      4.6
negative     4.2
Name: proportion, dtype: float64

── Electronics Corpus ──
Total reviews: 30,487
Unique products: 80
Echo / Tap: 4,270 reviews
Fire TV: 2,554 reviews

── Saved ──
sentiment_ready.csv
electronics_ready.csv


## EDA Summary

Three raw datasets were cleaned and standardized using a modular pipeline 
(load → select → clean per column). After removing duplicated rows, the sentiment corpus 
contains 39,794 reviews with a severe class imbalance: 91.2% positive, 4.6% 
neutral, 4.2% negative. This will be addressed in training via class weighting 
and a balanced evaluation set.

All three datasets are combined for sentiment training because sentiment is a 
domain-agnostic task — emotional language patterns generalize across product 
categories, and maximum review volume reduces imbalance in the minority classes. 
For clustering and text generation, only DS1 and DS3 are used: DS2 is almost 
entirely Electronics and adds no category diversity. DS3 provides the labeled 
primary_category column that serves as ground truth for clustering evaluation, 
while DS1 contributes additional review volume within the Electronics domain.

The electronics corpus contains 30,487 reviews across 80 unique products, 
organized into 5 meta-categories: Fire Tablets, Fire Kids Edition, Kindle 
E-Readers, Echo & Smart Speakers, and Fire TV & Streaming. Rather than forcing 
artificial diversity across sparse non-electronics categories, I decided to go 
deep within a single domain where sufficient product breadth and review volume 
exist to produce meaningful clustering and generation output. 

All processed files are saved to /data/processed/.